# 고급 모델링 기법 (Model Sub Classing)

본 튜토리얼에서는 고급 텐서플로우 모델링에 대하여 배웁니다.

**Model Sub Classing이란?**
- `tensorflow.keras.Model`을 class 상속 받아 모델을 구현하는 방법
- 객체 지향적인 설계 방법: Object Oriented Programming (OOP)
- 모델의 재사용성이 좋습니다 (OOP의 장점이기도 합니다.)

## 필요한 모듈 import

In [2]:
import tensorflow as tf

## 실습에 필요한 Dataset 로드

In [3]:
mnist = tf.keras.datasets.mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
x_train.shape, x_test.shape

((60000, 28, 28), (10000, 28, 28))

In [6]:
y_train.shape, y_test.shape

((60000,), (10000,))

**이미지 정규화 (Normalization)**

* 모든 이미지 픽셀(pixel)값들을 0~1 사이의 값으로 정규화 해 줍니다.
* x_train, x_valid 에 대해서만 정규화합니다.

In [7]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [8]:
x_train.min(), x_train.max()

(0.0, 1.0)

## Model Subclassing

- Model subclassing 구조를 만들기 위해서는 `tensorflow.keras.Model`을 상속받아야 합니다.
- `tensorflow.keras.Model`은 `tensorflow.keras.models.Model`을 레퍼런스 하고 있습니다.

In [9]:
class MyModel(tf.keras.Model):
    # 초기값 설정
    def __init__(self):
        super(MyModel, self).__init__()
    
    # 학습 함수 (fit)
    def call(self):
        pass

### 다음의 모델과 동일한 모델 구현

In [11]:
input_layer = tf.keras.Input(shape=(28, 28), name='InputLayer')
x1 = tf.keras.layers.Flatten(name='Flatten')(input_layer)
x2 = tf.keras.layers.Dense(256, activation='relu', name='Dense1')(x1)
x3 = tf.keras.layers.Dense(128, activation='relu', name='Dense2')(x2)
x4 = tf.keras.layers.Dense(64, activation='relu', name='Dense3')(x3)
x5 = tf.keras.layers.Dense(10, activation='softmax', name='OutputLayer')(x4)

func_model = tf.keras.Model(inputs=input_layer, outputs=x5, name='FunctionalModel')
func_model.summary()

Model: "FunctionalModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
InputLayer (InputLayer)      [(None, 28, 28)]          0         
_________________________________________________________________
Flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
Dense1 (Dense)               (None, 256)               200960    
_________________________________________________________________
Dense2 (Dense)               (None, 128)               32896     
_________________________________________________________________
Dense3 (Dense)               (None, 64)                8256      
_________________________________________________________________
OutputLayer (Dense)          (None, 10)                650       
Total params: 242,762
Trainable params: 242,762
Non-trainable params: 0
_____________________________________________

In [103]:
class MyModel(tf.keras.Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        # 초기값 설정
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(256, activation='relu')
        self.dense2 = tf.keras.layers.Dense(128, activation='relu')
        self.dense3 = tf.keras.layers.Dense(64, activation='relu')
        self.dense4 = tf.keras.layers.Dense(10, activation='softmax')
        
    # class overiding
    # 학습용 함수 정의
    # x는 input
    def call(self, x):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return x

In [104]:
mymodel = MyModel()

In [105]:
mymodel._name = 'subclass_model'

In [106]:
mymodel(tf.keras.layers.Input(shape=(28, 28)))

<tf.Tensor 'subclass_model/dense_51/Softmax:0' shape=(None, 10) dtype=float32>

In [107]:
mymodel.summary()

Model: "subclass_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_13 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_49 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_50 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_51 (Dense)             (None, 10)                650       
Total params: 242,762
Trainable params: 242,762
Non-trainable params: 0
_________________________________________________________________


## Model의 생성자 parameter 활용

In [108]:
class MyModel(tf.keras.Model):
    
    def __init__(self, units, num_classes):
        super(MyModel, self).__init__()
        # 초기값 설정
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units, activation='relu')
        self.dense2 = tf.keras.layers.Dense(units/2, activation='relu')
        self.dense3 = tf.keras.layers.Dense(units/4, activation='relu')
        self.dense4 = tf.keras.layers.Dense(num_classes, activation='softmax')
        
    # class overiding
    # 학습용 함수 정의
    # x는 input
    def call(self, x):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return x

In [109]:
mymodel = MyModel(256, 10)
mymodel(tf.keras.layers.Input(shape=(28, 28)))
mymodel.summary()

Model: "my_model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_14 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_53 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_54 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_55 (Dense)             (None, 10)                650       
Total params: 242,762
Trainable params: 242,762
Non-trainable params: 0
_________________________________________________________________


## 모델의 학습

In [110]:
mymodel.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
mymodel.fit(x_train, y_train, epochs=3)
mymodel.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2125 - accuracy: 0.9358
Epoch 2/3
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0924 - accuracy: 0.9715
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.0942 - accuracy: 0.9707


[0.09415823966264725, 0.9707000255584717]